In [1]:
# Произведем импорт необходимых библиотек

# Стандартныен библиотеки для работы с DF
import pandas as pd
import numpy as np
import datetime

# Библиотека для поиска сочетаний (регуляров) 
import re

# Библиотеки для парсинга
from bs4 import BeautifulSoup
import requests

# Библиотека для кодирования 
import category_encoders as ce

# Библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

# Импорт модуля для предварительной обработки данных
from sklearn import preprocessing

# Используется для подсчета времени обучения в разделе 
# "Модель машинного обучения"
from timeit import default_timer as timer
from datetime import timedelta

# Библиотека и методы для определения эмоционального настроя текста
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nitys\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Чтение DF
hotels_t = pd.read_csv('data/hotels_test.csv')

In [3]:
# Проверка данных
hotels_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

Имеются пропуски по lat и lng. Ручной метод, примененный изначально применять не будем. 
Пропишем медианную по стране. 
Для начала сделаем преобразование признаков, создавая признак страны.  

In [4]:
# Пропишем функцию для извлечения страны из признака адреса отеля
def country_extract(string):
    country = list(string.split(' '))[-1]
    return country

# Создадим новый признак страны отеля, применив функцию к адресу
hotels_t['country'] = hotels_t['hotel_address'].apply(country_extract)

# По результату получили одно из значений 'Kingdom', 
# Напишем функцию, дающую полное название страны
hotels_t['country'] = hotels_t['country'].apply(
    lambda x: 'United Kingdom' if x == 'Kingdom' else x
    )

# Удалим признак адреса, он для модели не пригоден 
hotels_t = hotels_t.drop('hotel_address', axis=1)


In [5]:
# Создание DF с списком стран и средним значением широты
data_lat = pd.DataFrame(
    hotels_t.groupby('country')['lat'].mean())

# Создание DF с списком стран и средним значением долготы
data_lng = pd.DataFrame(
    hotels_t.groupby('country')['lng'].mean())

# Заполним пропуски словами-маркерами, по ним будем перезаполнять
hotels_t['lat'] = hotels_t['lat'].fillna('toreplacelat')
hotels_t['lng'] = hotels_t['lng'].fillna('toreplacelng')

# Переименуем оригинальные названия признаков 
# В дальнейшем их удалим, после создания новых
hotels_t = hotels_t.rename(columns={'lat': 'lat_orig', 'lng': 'lng_orig'})

# Функция заполнения пропусков по "маркерам" - долгота
# Если слово маркер - заменяем средним по стране, 
# Если признак заполнен, оставляем его значение
def lat_filling(col_country, col_lat):
    if col_lat == 'toreplacelat':
        result = data_lat.loc[f'{col_country}'][0]
    else:
        result = col_lat
    return result

# Создание признака долготы
hotels_t['lat'] = hotels_t.apply(
    lambda x: lat_filling(x.country, x.lat_orig), axis=1
    )

# Функция заполнения пропусков по "маркерам" - широта
# Если слово маркер - заменяем средним по стране, 
# Если признак заполнен, оставляем его значение
def lng_filling(col_country, col_lng):
    if col_lng == 'toreplacelng':
        result = data_lng.loc[f'{col_country}'][0]
    else:
        result = col_lng
    return result

# Создание признака широты
hotels_t['lng'] = hotels_t.apply(
    lambda x: lng_filling(x.country, x.lng_orig), axis=1
    )

# Удаляем ненужные признаки, в которых есть пропуски
hotels_t = hotels_t.drop(['lat_orig', 'lng_orig'], axis=1)

C:\Users\nitys\AppData\Local\Temp\ipykernel_141852\4061079087.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result = data_lat.loc[f'{col_country}'][0]
C:\Users\nitys\AppData\Local\Temp\ipykernel_141852\4061079087.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result = data_lng.loc[f'{col_country}'][0]


Пропуски заполнены, вернемся к исходной последовательности преобразования DF

In [6]:
# Произведем чтение DF
url='https://drive.google.com/file/d/1ATDWtM5RP-OtiFLUuePr5VGzcP3Cpdtx/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
top_50_world_df = pd.read_csv(url, encoding='latin-1')

# Пустой список, куда занесем отдельные слова из названий отелей
top_50_world_list = []

# Функция для извлечения слов из названий отелей
def top_50_name_extract(name):
    words = str(name).split(' ')
    for word in words:
        top_50_world_list.append(word.lower())

# Применим функцию    
top_50_world_df['Name'].apply(top_50_name_extract)

# Преобразуем список в множество
top_50_world_set = set(top_50_world_list)

In [7]:
# Функция для парсинга сайта, маскируемся под браузер для обхода блокировки
def get_page_contents(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'}
    page = requests.get(url, headers=headers)
    return BeautifulSoup(page.text, 'html.parser')

# Адрес интересующей нас страницы
url = 'https://www.tripadvisor.com/TravelersChoice-Hotels-cTop-g4.html'

# Создание объект beatiful soup
soup = get_page_contents(url)

# Создание предобработанного списка, в элементах которого будут 
# содержатся названия отелей
top_25_dirty_list = str(soup).split('</h2')

# Счетчик итераций (в первом элементе в отличии от всех остальных элементов 
# отличается регулярное выражения для выборки названия отеля)
counter = 0

# Пустой список, будем наполнять названиями отелей
top_25_tripadv = []

# Циклом вычленяем названия отелей из предообработанного списка 
for elem in top_25_dirty_list:
    
    # Обработчик исключений, для успешного выполнения цикла
    try:
        if counter == 0:
            name = re.search('eIegw">(.*)', str(elem))
        else:
            name = re.search('rRtyp">(.*)', str(elem))
        name = name.group(1)
        top_25_tripadv.append(name)
    except:
        continue
    counter += 1
    
# Пустой список, куда занесем отдельные слова из названий отелей
top_25_europe_list = []

# Циклом вычленяем отдельные слова из названий отелей 
for name in top_25_tripadv:
    
    words = name.split(' ')
    for word in words:
        top_25_europe_list.append(word.lower())

# Преобразуем список в множество
top_25_europe_set = set(top_25_europe_list)

In [8]:
# Ссылка на сформированный файл
url='https://drive.google.com/file/d/1yf85a0arfHVMmU3jqOpua7Ov0FtYjpTa/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

# Создание DF из файла
chain_hotels = pd.read_csv(url, sep=';')

# Список названий отелей из DF
chain_hotels_list_orig = list(chain_hotels['Managed brands'])

# Пустой список, куда занесем отдельные слова из названий отелей
chain_hotels_list = []

# Вычленим циклом слова из названий отелей и занесем их с список выше
for name in chain_hotels_list_orig:
    # Обработчик исключений требуется для избежания вызова ошибок при обработки 
    # (имеются числовые данные и символы в результате преобразования)
    try:
        words = list(name.split(' '))
        for word in words: 
            chain_hotels_list.append(word.lower())
    except:
        continue

# Преобразуем список в множество
chain_hotels_set = set(chain_hotels_list)

In [9]:
# Объединим множества 

key_words_top = top_50_world_set.union(top_25_europe_set)
key_words_top = key_words_top.union(chain_hotels_set)

In [10]:
# Создадим список слов к удалению из множества
# Данные слова/артикли могут быть общими как для брендового отеля, 
# так и для обычного, так же, внесем названия столиц, представленных стран
excluding_words = {'hotel', 'spa', 'resort', 'inn', 'ink', 'apartment', 
                   'apart', 'villa', 'hostel', 'motel', 'bed', 'breakfast', 
                   'b&b', 'palace', 'park',
                   
                   'the', 'by', 'a', 'an', 'of', 'in', 'la', 'les', 'le',
                   
                   'london', 'paris', 'amsterdam', 'madrid', 'barcelona', 
                   'vienna', 'rome'
                   }

key_words = key_words_top.difference(excluding_words)

In [11]:
# Функция проверки вхождения множества слов из названия отеля в 
# ранее созданное множество слов характерных для сетевых или топовых отелей
# Функция возвращает 1 - если есть вхождение, в противном случае - 0.
def hotel_name_top(name):
    words = name.split(' ')
    name_set = set()
    
    for word in words:
        name_set.add(word.lower())
    
    if name_set.intersection(key_words):
        return 1
    else:
        return 0


# Создание нового бинарного признака является ли отель сетевым/топовым
hotels_t['top'] = hotels_t['hotel_name'].apply(hotel_name_top)

# Удалим признак названия отеля
hotels_t = hotels_t.drop(['hotel_name'], axis =1)

In [12]:
# Преобразуем тип признака в datetime, воспользуемся аксессором по месяцу
hotels_t['review_date'] = pd.to_datetime(hotels_t['review_date'])
hotels_t['review_date'] = hotels_t['review_date'].dt.month
hotels_t['review_date'] = hotels_t['review_date'].astype('int64')

# Функция преобразования месяца в сезон
def season_detect(month):
    if month in [12, 1, 2]:
        return 'winter'
    if month in [3, 4, 5]:
        return 'spring'
    if month in [6, 7, 8]:
        return 'summer'
    if month in [9, 10, 11]:
        return 'autumn'

# Создание нового признака сезона отзыва постояльца
hotels_t['review_season'] = hotels_t['review_date'].apply(season_detect)

# Удалим уже ненужный признак
hotels_t = hotels_t.drop(['review_date'], axis=1)

In [13]:
# Функция преобразования признака в категориальный
def days_convert(feature):
    days = int(feature.split(' ')[0])
    if 0 <= days < 100:
        return '0-100' 
    if 100 <= days < 200:
        return '100-200' 
    if 200 <= days < 300:
        return '100-300' 
    if 300 <= days < 400:
        return '300-400' 
    if 400 <= days < 500:
        return '400-500' 
    if 500 <= days < 600:
        return '500-600' 
    if 600 <= days:
        return '<_600' 

# Создание нового категориального признака времени, прошедшего с обзора
hotels_t['days_since_review'] = hotels_t['days_since_review'].apply(
    days_convert
    )

In [14]:
# Обработаем исходный признак, удалив лишние пробелы в строковых данных
hotels_t['reviewer_nationality'] = hotels_t['reviewer_nationality'].apply(
    lambda x: x.strip()
    )

# Функция для сравнения значений двух признаков
# Возвращает 1, если совпадение, в противном случае 0
def compare_country(col_1, col_2):
    if str(col_1) == str(col_2):
        return 1
    else:
        return 0

# Создаем новый признак, свидетельствующий о равности гражданства и страны отеля
hotels_t['same_country'] = hotels_t.apply(
    lambda x: compare_country(x.reviewer_nationality, x.country), axis = 1
    ) 

# Удаляем признак гражданства постояльца
hotels_t = hotels_t.drop(['reviewer_nationality'], axis = 1)

In [15]:
"""

# Создадим новые признаки - копии существующих и с ними будем работать
hotels_t[['positive_rev_to_vader', 'negative_rev_to_vader']] = \
    hotels_t[['positive_review', 'negative_review']].copy()

# Функция для подмены позитивных слов в негативных отзывах
def negative_turnover(elem):
    neg_to_pos_list = ['leaving', 'no negative', 'nothing']
    for word in neg_to_pos_list:
        elem = elem.lower()
        elem = elem.replace(f'{word}', 'positive')
    return elem

# Функция для подмены негативных слов в позитивных отзывах
def positive_turnover(elem):
    pos_to_neg_list = ['nothing', 'no positive']
    for word in pos_to_neg_list:
        elem = elem.lower().replace(f'{word}', 'negative')
    return elem
    
# Применений функций к признакам
hotels_t['negative_rev_to_vader'] = \
    hotels_t['negative_rev_to_vader'].apply(negative_turnover)


hotels_t['positive_rev_to_vader'] = \
    hotels_t['positive_rev_to_vader'].apply(positive_turnover)

# "Склейка" позитивного и негативного отзывов в единый (для извлечений compound)
hotels_t['to_vader'] = \
    hotels_t['positive_rev_to_vader'] + hotels_t['negative_rev_to_vader'] 

# Функция для получения коэффициента общего значения 
# по словам суммарного отзыва
def vader_analitics(review):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(review)
    return ss['compound']

# Создание нового признака 
hotels_t['vader_score'] = hotels_t['to_vader'].apply(vader_analitics)

"""

'\n\n# Создадим новые признаки - копии существующих и с ними будем работать\nhotels_t[[\'positive_rev_to_vader\', \'negative_rev_to_vader\']] =     hotels_t[[\'positive_review\', \'negative_review\']].copy()\n\n# Функция для подмены позитивных слов в негативных отзывах\ndef negative_turnover(elem):\n    neg_to_pos_list = [\'leaving\', \'no negative\', \'nothing\']\n    for word in neg_to_pos_list:\n        elem = elem.lower()\n        elem = elem.replace(f\'{word}\', \'positive\')\n    return elem\n\n# Функция для подмены негативных слов в позитивных отзывах\ndef positive_turnover(elem):\n    pos_to_neg_list = [\'nothing\', \'no positive\']\n    for word in pos_to_neg_list:\n        elem = elem.lower().replace(f\'{word}\', \'negative\')\n    return elem\n    \n# Применений функций к признакам\nhotels_t[\'negative_rev_to_vader\'] =     hotels_t[\'negative_rev_to_vader\'].apply(negative_turnover)\n\n\nhotels_t[\'positive_rev_to_vader\'] =     hotels_t[\'positive_rev_to_vader\'].apply(pos

In [16]:
"""

# Функция для получения коэффициента позитивного окраса 
# по словам позитивного отзыва
def vader_analitics_positive(review):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(review)
    return ss['pos']

# Создание нового признака 
hotels_t['vader_score_pos'] = hotels_t['positive_rev_to_vader'].apply(vader_analitics_positive)

"""

"\n\n# Функция для получения коэффициента позитивного окраса \n# по словам позитивного отзыва\ndef vader_analitics_positive(review):\n    sid = SentimentIntensityAnalyzer()\n    ss = sid.polarity_scores(review)\n    return ss['pos']\n\n# Создание нового признака \nhotels_t['vader_score_pos'] = hotels_t['positive_rev_to_vader'].apply(vader_analitics_positive)\n\n"

In [17]:
"""

# Функция для получения коэффициента негативного окраса 
# по словам негативного отзыва
def vader_analitics_negative(review):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(review)
    return ss['neg']

# Создание нового признака 
hotels_t['vader_score_neg'] = hotels_t['negative_rev_to_vader'].apply(vader_analitics_negative)

"""

"\n\n# Функция для получения коэффициента негативного окраса \n# по словам негативного отзыва\ndef vader_analitics_negative(review):\n    sid = SentimentIntensityAnalyzer()\n    ss = sid.polarity_scores(review)\n    return ss['neg']\n\n# Создание нового признака \nhotels_t['vader_score_neg'] = hotels_t['negative_rev_to_vader'].apply(vader_analitics_negative)\n\n"

In [18]:
"""

# Список признаков к копированию 
# (коэффциентов от работы метода SentimentIntensityAnalyzer)
verdered_feats_list = ['vader_score', 'vader_score_pos', 'vader_score_neg']

# Выведем полученные значения 

vader_scores_t = hotels_t[verdered_feats_list].copy()
vader_scores_t.to_csv('data/vader_scores_t.csv', index=False)

"""

"\n\n# Список признаков к копированию \n# (коэффциентов от работы метода SentimentIntensityAnalyzer)\nverdered_feats_list = ['vader_score', 'vader_score_pos', 'vader_score_neg']\n\n# Выведем полученные значения \n\nvader_scores_t = hotels_t[verdered_feats_list].copy()\nvader_scores_t.to_csv('data/vader_scores_t.csv', index=False)\n\n"

In [19]:
url = 'https://drive.google.com/file/d/1QOOTUIgujpAeo3vQumlzgqihWiYRHK2a/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

vader = pd.read_csv(url)

hotels_t = pd.concat([hotels_t, vader], axis=1)

In [21]:
# Создадим копии признаков, которые будем обрабатывать
hotels_t['negative_review_copy'] = hotels_t['negative_review']
hotels_t['positive_review_copy'] = hotels_t['positive_review']

# Произведем удаление значений, не характеризующих негативный 
# или позитивный опыт
hotels_t['negative_review_copy'] = \
    hotels_t['negative_review_copy'].apply(
    lambda x: '' if (x.strip().lower() == 'leaving') |
    (x.strip().lower() == 'no negative')
    else x
    )

hotels_t['positive_review_copy'] = \
    hotels_t['positive_review_copy'].apply(
    lambda x: '' if (x.strip().lower() == 'no positive') else x
    )
    
# Список частиц, не участвующих в подсчете слов, так же, 
# сюда добавим пустое значение, оно местами присутствует в DF, 
# и, вероятно, в оригинальном датасете и давало лишние числа
article_list = ['the', 'a', 'an', 'and', 'of', 'in', 'on', 'or', '']

# Функция подсчета слов
def word_counter(rev):
    
    result = []
    
    rev = rev.strip().lower()
    rev_list = rev.split(' ')
    
    for elem in rev_list:
        if elem not in article_list:
            result.append(elem)
    
    return len(result)

# Создадим новые признаки количества слов, применив функцию
hotels_t['negative_word_true'] = \
    hotels_t['negative_review_copy'].apply(word_counter)

hotels_t['positive_word_true'] = \
    hotels_t['positive_review_copy'].apply(word_counter)

# Функция расчета соотношения позитивных слов к негативным
def ratio_calc(negative, positive):
    try:
        result = positive / negative
    except:
        result = (positive + 1) / (negative + 1)
        
    return result

# Создаем новый признак соотношения слов, применив функцию
hotels_t['pos_neg_ratio'] = hotels_t.apply(
    lambda x: ratio_calc(x.negative_word_true,
                         x.positive_word_true),
    axis = 1
    )

# Список признаков на удаление
list_to_drop_revs = ['negative_review_copy', 'positive_review_copy', 
                'negative_review', 'positive_review', 
                'review_total_negative_word_counts', 
                'review_total_positive_word_counts'
                ]

# Удаление признаков
hotels_t = hotels_t.drop(
    columns = list_to_drop_revs, axis=1
    )

# Переименуем колонки новых признаков
hotels_t = hotels_t.rename(columns={
    'negative_word_true' : 'negative_word_qnt',
    'positive_word_true' : 'positive_word_qnt'
    })

In [22]:
# Функция для очистки значений признака тагов от лишних элементов 
def tag_renewal(tag):

    tag = tag.split(',')

    # Формирование списка из очищенных тагов
    new_tag = list(map(
        lambda x: ( re.search( "' (.*) '", str(x) ) ).group(1).lower(), 
        tag
        ))
   
    # Возврат обновленного очищенного списка
    return new_tag

# Создаем новый признак, очищенную копию прошлого признака 
hotels_t['new_tags'] = hotels_t['tags'].apply(tag_renewal)

# Удалим отработанный признак
hotels_t = hotels_t.drop(columns=['tags'], axis=1)

In [23]:
# Ряд типовых функция для обработки признака с возвратом бинарного значения
def is_mobile(feature):
    if re.findall('mobile device', str(feature)):
        return 1
    else:
        return 0

def is_leisure(feature):
    if re.findall('leisure trip', str(feature)):
        return 1
    else:
        return 0

def is_business(feature):
    if re.findall('business trip', str(feature)):
        return 1
    else:
        return 0

# Создание новых признаков с применением написанных функций
hotels_t['mobile_submit'] = hotels_t['new_tags'].apply(is_mobile)
hotels_t['leisure trip'] = hotels_t['new_tags'].apply(is_leisure)
hotels_t['business trip'] = hotels_t['new_tags'].apply(is_business)

In [24]:
# Обработка в части категориальных признаков будет связана с индексами 
# элементов списка - значения признака new_tags, поэтому, 
# необходимо сделать проверку на количество элементов списка, для 
# этого создадим вспомогательный признак - количество тагов .
hotels_t['tags_qnt'] = hotels_t['new_tags'].apply(
    lambda x: len(x)
    )

In [25]:
# Функция для извлечения описание типа номера 
# (Применяем к тем строкам, где высока вероятность соблюдения порядка
# в тэгах, задаемся количеством тэгов больше или равно 4)
def room_extract(qnt, tag):
    if qnt >= 4:
        return tag[2]
    else: 
        return 'Unknown'

# Применим функцию к DF для создания временного признака с типом номера
hotels_t['room_type_temp'] = hotels_t.apply(
    lambda x: room_extract(x.tags_qnt, x.new_tags), axis = 1
    ) 

In [26]:
# На основе выбранных ключевых слов - функция для 
# определения категории номера 
def room_type(tags):
    tags = str(tags).lower()
    
    if re.findall('superior', tags) or \
        re.findall('deluxe', tags) or \
            re.findall('queen room', tags) or \
                re.findall('club', tags):
                    return 'superior'
    if re.findall('suite', tags):
        return 'suite'
    else:
        return 'standart'

# Создаем новый признак типа номера (он уже будет постоянный)
hotels_t['room_type'] = hotels_t['new_tags'].apply(room_type)

# Удалим временный признак типа номера 
hotels_t = hotels_t.drop(columns = ['room_type_temp'], axis=1)

In [27]:
# Аналогичный подход к обработке признака состава путешественников

# Функция для извлечения описание состава путешественников 
# (Применяем к тем строкам, где высока вероятность соблюдения порядка
# в тэгах, задаемся количеством тэгов больше или равно 4)
def travelers_extract(qnt, tag):
    if qnt >= 4:
        return tag[1]
    else: 
        return 'Unknown'

# Применим функцию к DF для создания временного признака с типом номера
hotels_t['travelers_temp'] = hotels_t.apply(
    lambda x: travelers_extract(x.tags_qnt, x.new_tags), axis = 1
    )

In [28]:
# На основе выбранных ключевых слов - функция для 
# определения категории номера 
def travelers_type(tags):
    tags = str(tags).lower()
    
    if re.findall('solo', tags):
        return 'solo'
    if re.findall('group', tags) or \
        re.findall('friend', tags):
        return 'friends'
    if re.findall('family', tags) or \
        re.findall('child', tags):
        return 'family'
    if re.findall('couple', tags):
        return 'couple'
    else:
        return None

# Создаем новый признак типа номера (он уже будет постоянный)
hotels_t['traveler_type'] = hotels_t['new_tags'].apply(travelers_type)

# Удалим временный признак типа номера 
hotels_t = hotels_t.drop(columns = ['travelers_temp'], axis=1)

In [29]:
# Функция для поиска значений количества ночей
def nights_calc(tags):
    try:
        tags = str(tags).lower()
        result = re.search('stayed (.*) night', tags).group(1)
        return int(result)
    except:
        return None

hotels_t['nights_qnt'] = hotels_t['new_tags'].apply(nights_calc)        

In [30]:
# Количество пропусков не вилико, заполним их медианой по признаку
hotels_t['nights_qnt'] = hotels_t['nights_qnt'].fillna(
    hotels_t['nights_qnt'].median()
    )

In [31]:
hotels_t['nights_qnt'] = hotels_t['nights_qnt'].apply(
    lambda x: 10 if x >= 10 else x
    ) 

In [32]:
hotels_t = hotels_t.drop(columns = ['new_tags', 'tags_qnt'], axis=1)

In [33]:
hotels_t_encode = hotels_t.copy()

In [34]:
# Список признаков для кодирования
columns_to_encode = ['days_since_review', 
                     'country',
                     'review_season', 
                     'traveler_type'
                     ]

# Циклом произведем кодирование признаков
for feature in columns_to_encode:
    encoder = ce.OneHotEncoder(cols=[f'{feature}'], use_cat_names=True)
    type_bin = encoder.fit_transform(hotels_t_encode[f'{feature}'])
    hotels_t_encode = pd.concat([hotels_t_encode, type_bin], axis=1)

room_type_dict = {
    'standart' : 1,
    'superior' : 2,
    'suite' : 3
    }

# Порядковое кодирование признака room_type
ord_encoder = ce.OrdinalEncoder(mapping=[{
    'col': 'room_type',
    'mapping': room_type_dict
    }])

data_bin = ord_encoder.fit_transform(hotels_t_encode['room_type'])

hotels_t_encode = pd.concat(
    # Сразу "по месту" произведем удаление исходного признака room_type
    [hotels_t_encode.drop(columns=['room_type'], axis=1), data_bin], 
    axis=1
    )

In [35]:
# Произведем удаление признаков, которые уже закодированы
hotels_t_encode = hotels_t_encode.drop(
    columns = columns_to_encode,
    axis = 1)

In [39]:
# инициализируем стандартизатор StandardScaler
r_scaler = preprocessing.RobustScaler()

# трансформируем исходный датасет (закодированный)
hotels_t_encode_norm = r_scaler.fit_transform(hotels_t_encode)

# Преобразуем промежуточный датасет в полноценный датафрейм
hotels_t_encode_norm = pd.DataFrame(
    hotels_t_encode_norm, 
    columns = list(hotels_t_encode.columns)
    )

In [40]:
# инициализируем стандартизатор StandardScaler
s_scaler = preprocessing.StandardScaler()

# трансформируем исходный датасет
hotels_t_encode_norm_stnd = s_scaler.fit_transform(hotels_t_encode_norm)

# Преобразуем промежуточный датасет в полноценный датафрейм для визуализации
hotels_t_encode_norm_stnd = pd.DataFrame(
    hotels_t_encode_norm_stnd, 
    columns = list(hotels_t_encode.columns)
    )

In [41]:
# инициализируем стандартизатор StandardScaler
s_scaler = preprocessing.StandardScaler()

# трансформируем исходный датасет
hotels_t_encode_norm_stnd = s_scaler.fit_transform(hotels_t_encode_norm)

# Преобразуем промежуточный датасет в полноценный датафрейм для визуализации
hotels_t_encode_norm_stnd = pd.DataFrame(
    hotels_t_encode_norm_stnd, 
    columns = list(hotels_t_encode.columns)
    )

In [42]:
df = hotels_t_encode_norm_stnd.copy()

In [ ]:
# Делим данные на прогнозируемые значения и остальные
X = df.drop(['reviewer_score'], axis = 1)  
y = df['reviewer_score'] 

# Метод для деления данных на категории
from sklearn.model_selection import train_test_split  

# 4 категории 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
    )

# инструмент для создания и обучения модели  
from sklearn.ensemble import RandomForestRegressor 

# инструменты для оценки точности модели 
from sklearn import metrics 

# модель
regr = RandomForestRegressor(n_estimators=100)  
    
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
    
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

# Коэфициент оценки эффективности
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)

print('MAPE:', mape)

In [44]:
url = 'https://drive.google.com/file/d/1GgW9N_C6fqoXUD_P4pdIUQPcQg-ulKyd/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

url

'https://drive.google.com/uc?id=1GgW9N_C6fqoXUD_P4pdIUQPcQg-ulKyd'